# Compute trajectories for Greenland

In [7]:
import numpy as np
import pylab as plt
import xarray as xr
from pismragis.trajectories import compute_trajectory, compute_perturbation, get_perturbed_velocities
from joblib import Parallel, delayed
import geopandas as gp
from pathlib import Path
from pismragis.processing import tqdm_joblib
import pandas as pd
from pyDOE import lhs
from scipy.stats.distributions import uniform
from tqdm.auto import tqdm

In [8]:
data_url = Path("~/Google Drive/My Drive/data/ITS_LIVE/GRE_G0240_0000.nc")
ogr_url = Path("/Users/andy/Google Drive/My Drive/data/GreenlandFluxGatesAschwanden/greenland-flux-gates-jibneighbors.shp")

In [9]:
n_draw_samples = 10
unif_sample = lhs(2, n_draw_samples)

In [10]:
with tqdm_joblib(tqdm(desc="Processing perturbations", total=n_draw_samples, position=0, leave=True)) as progress_bar:
    all_perturb_glaciers = Parallel(n_jobs=10)(
        delayed(compute_perturbation)(data_url, ogr_url,
                                       perturbation=perturb, sample=unif_sample[perturb, :], 
                                       total_time=1_000, dt=1, reverse=True)
                for perturb in range(n_draw_samples))
    del progress_bar

all_perturb_glaciers = pd.concat(all_perturb_glaciers).reset_index(drop=True)

Processing perturbations:   0%|          | 0/10 [00:00<?, ?it/s]

Integrating trajectory:  41%|████      | 412/1000 [00:00<00:00, 1012.66it/s]
                                                                            
Integrating trajectory: 100%|█████████▉| 997/1000 [00:00<00:00, 1039.11it/s]
                                                                            
Integrating trajectory:  79%|███████▉  | 788/1000 [00:00<00:00, 1075.94it/s]
                                                                            
Integrating trajectory:  84%|████████▍ | 840/1000 [00:00<00:00, 1062.65it/s]
                                                                            
Integrating trajectory:  95%|█████████▌| 951/1000 [00:00<00:00, 1075.31it/s]
                                                                            
Integrating trajectory:  22%|██▏       | 218/1000 [00:00<00:00, 1093.72it/s]
                                                                            
Integrating trajectory:  57%|█████▋    | 574/1000 [00:00<00:00, 1129.80it/s]

Integrating trajectory:  10%|█         | 105/1000 [00:00<00:00, 1048.52it/s]
                                                                            
Integrating trajectory:  23%|██▎       | 229/1000 [00:00<00:00, 1098.99it/s]
                                                                            
Integrating trajectory:  55%|█████▍    | 548/1000 [00:00<00:00, 1066.03it/s]
                                                                            
Integrating trajectory:  99%|█████████▉| 988/1000 [00:00<00:00, 1080.31it/s]
                                                                            
Integrating trajectory:   0%|          | 0/1000 [00:00<?, ?it/s]
                                                                            
Integrating trajectory:  89%|████████▉ | 888/1000 [00:00<00:00, 1086.01it/s]
                                                                           
Integrating trajectory:  22%|██▎       | 225/1000 [00:00<00:00, 1048.17it/s]
            

Integrating trajectory: 100%|█████████▉| 995/1000 [00:00<00:00, 1074.10it/s]
                                                                            
Integrating trajectory:  55%|█████▍    | 546/1000 [00:00<00:00, 1040.59it/s]
                                                                            
Integrating trajectory:  34%|███▎      | 335/1000 [00:00<00:00, 1103.76it/s]

Integrating trajectory:  53%|█████▎    | 534/1000 [00:00<00:00, 1072.23it/s]
                                                                            
Integrating trajectory:  88%|████████▊ | 884/1000 [00:00<00:00, 979.44it/s] 
                                                                            
Integrating trajectory:   0%|          | 0/1000 [00:00<?, ?it/s]
                                                                           
Integrating trajectory:  87%|████████▋ | 867/1000 [00:00<00:00, 989.09it/s]
                                                                           
             

Integrating trajectory:   0%|          | 0/1000 [00:00<?, ?it/s]
                                                                            
Integrating trajectory:  99%|█████████▉| 988/1000 [00:00<00:00, 1050.67it/s]
                                                                            
Integrating trajectory:  21%|██        | 209/1000 [00:00<00:00, 1055.43it/s]
                                                                            
Integrating trajectory:  44%|████▍     | 440/1000 [00:00<00:00, 1113.53it/s]

Integrating trajectory:  65%|██████▍   | 649/1000 [00:00<00:00, 1095.84it/s]
                                                                            
Integrating trajectory:  50%|█████     | 505/1000 [00:00<00:00, 1010.15it/s]
                                                                            
Integrating trajectory:  67%|██████▋   | 674/1000 [00:00<00:00, 1146.10it/s]
                                                                            
Integratin

...

Integrating trajectory:  57%|█████▋    | 573/1000 [00:00<00:00, 914.42it/s]
                                                                           
Integrating trajectory:  60%|█████▉    | 595/1000 [00:00<00:00, 948.68it/s]
                                                                           
Integrating trajectory:  99%|█████████▉| 989/1000 [00:01<00:00, 949.98it/s]
                                                                           
Integrating trajectory:  89%|████████▉ | 890/1000 [00:00<00:00, 969.74it/s]

Integrating trajectory:   0%|          | 0/1000 [00:00<?, ?it/s]
                                                                           
Integrating trajectory:   0%|          | 0/1000 [00:00<?, ?it/s]

Integrating trajectory:   0%|          | 0/1000 [00:00<?, ?it/s]
                                                                           
Integrating trajectory:   9%|▉         | 91/1000 [00:00<00:01, 900.98it/s]

Integrating trajectory: 100%|█████████▉| 99

Integrating trajectory:  39%|███▉      | 389/1000 [00:00<00:00, 941.01it/s]
                                                                           
Integrating trajectory:  36%|███▋      | 364/1000 [00:00<00:00, 870.28it/s]
                                                                           
Integrating trajectory: 100%|█████████▉| 998/1000 [00:01<00:00, 838.45it/s]
                                                                           
Integrating trajectory:  61%|██████▏   | 614/1000 [00:00<00:00, 840.27it/s]
                                                                           
Integrating trajectory:   0%|          | 0/1000 [00:00<?, ?it/s]
                                                                           
Integrating trajectory:  85%|████████▍ | 847/1000 [00:00<00:00, 902.68it/s]
                                                                           
Integrating trajectory:  95%|█████████▌| 950/1000 [00:01<00:00, 930.50it/s]

Integrating trajectory

Integrating trajectory:  48%|████▊     | 478/1000 [00:00<00:00, 906.52it/s]
                                                                           
Integrating trajectory:  79%|███████▉  | 790/1000 [00:00<00:00, 972.64it/s]
                                                                           
Integrating trajectory:  67%|██████▋   | 668/1000 [00:00<00:00, 974.31it/s]
                                                                           
                                                                           
Integrating trajectory:  69%|██████▉   | 693/1000 [00:00<00:00, 909.43it/s]
                                                                           
Integrating trajectory:  81%|████████  | 807/1000 [00:00<00:00, 965.48it/s]
                                                                           
Integrating trajectory:   0%|          | 0/1000 [00:00<?, ?it/s]
                                                                           
Integrating trajectory:

Integrating trajectory:  57%|█████▋    | 566/1000 [00:00<00:00, 902.77it/s]
                                                                           
Integrating trajectory:  76%|███████▌  | 761/1000 [00:00<00:00, 938.96it/s]
                                                                           
Integrating trajectory:   0%|          | 0/1000 [00:00<?, ?it/s]
                                                                           
Integrating trajectory:  96%|█████████▌| 959/1000 [00:01<00:00, 960.61it/s]
                                                                           
Integrating trajectory:  71%|███████   | 711/1000 [00:00<00:00, 985.24it/s]
                                                                           
Integrating trajectory:   0%|          | 0/1000 [00:00<?, ?it/s]
                                                                           
Integrating trajectory:  19%|█▊        | 186/1000 [00:00<00:00, 933.83it/s]
                                  

Integrating trajectory:  97%|█████████▋| 970/1000 [00:01<00:00, 948.97it/s]
                                                                           
Integrating trajectory:   0%|          | 0/1000 [00:00<?, ?it/s]
                                                                           
Integrating trajectory:  20%|█▉        | 196/1000 [00:00<00:00, 981.42it/s]

Integrating trajectory:  98%|█████████▊| 976/1000 [00:01<00:00, 977.34it/s]
                                                                           
Processing Sermeq Kujalleq: 100%|██████████| 6/6 [00:19<00:00,  3.27s/it]s]

Integrating trajectory:  57%|█████▋    | 573/1000 [00:00<00:00, 956.36it/s]

Integrating trajectory:  78%|███████▊  | 781/1000 [00:00<00:00, 990.43it/s]
                                                                           
Integrating trajectory:  89%|████████▉ | 890/1000 [00:00<00:00, 982.53it/s] 
                                                                            
                  

In [ ]:
from tqdm.auto import trange